In [8]:
import simpy
import random 
import numpy as np

In [9]:
class JusticeSystem():
    def __init__(self, env) -> None:
        self.active_cases = 0
        self.env = env
    # The summoned process
    def summoned(self, env):
        while True:
            self.active_cases += 1
            print(f'At day {env.now+1}, the number of active cases is {self.active_cases},000 cases')
            yield self.env.timeout(1)

In [10]:
def simulate(env):
    justice_system = JusticeSystem(env)
    env.process(justice_system.summoned(env)) #Add process `summoned` to the simpy env
    yield env.timeout(0)

In [11]:
env = simpy.Environment()

env.process(simulate(env)) # Add process `simulate` to the simpy env
env.run(until=100)

At day 1, the number of active cases is 1,000 cases
At day 2, the number of active cases is 2,000 cases
At day 3, the number of active cases is 3,000 cases
At day 4, the number of active cases is 4,000 cases
At day 5, the number of active cases is 5,000 cases
At day 6, the number of active cases is 6,000 cases
At day 7, the number of active cases is 7,000 cases
At day 8, the number of active cases is 8,000 cases
At day 9, the number of active cases is 9,000 cases
At day 10, the number of active cases is 10,000 cases
At day 11, the number of active cases is 11,000 cases
At day 12, the number of active cases is 12,000 cases
At day 13, the number of active cases is 13,000 cases
At day 14, the number of active cases is 14,000 cases
At day 15, the number of active cases is 15,000 cases
At day 16, the number of active cases is 16,000 cases
At day 17, the number of active cases is 17,000 cases
At day 18, the number of active cases is 18,000 cases
At day 19, the number of active cases is 19,00